In [2]:
from numpy import *
import pandas as pd

from bs4 import BeautifulSoup
from operator import itemgetter
from pprint import pprint
import requests

# Make Product DF 

### The below code is to make a dataframe out of every product of every restaurant that can deliver to CEU
#### The code is to be debugged so that it runs for all restaurants + We might need to add more restaurants so that we can have 20 non-Budapest based restaurantes to comply with the exercise

In [7]:
link_base = 'https://www.netpincer.hu/'

#Parse netpincer to get list of html links for restaurants that deliver to CEU
response = requests.get('https://www.netpincer.hu/restaurants/new?lat=47.501185&lng=19.049364&vertical=restaurants')
soup = BeautifulSoup(response.content, "html.parser") 
#Find all restaurant link in html
restaurants = soup.find('ul', {'class':'vendor-list'}).findAll('a')
#create list of links to iterate over and parse them 
link_list = [link_base+link.get('href') for link in restaurants]

In [266]:
final_df = pd.DataFrame(columns = ['Restaurant', 'Product_Name', 'Price']) 

In [12]:
def netpincer_parser(URL): 
    global final_df
    prices = []
    
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, "html.parser") 

    #Getting name of the restaurant
    restaurant = soup.find('div',{'class':'vendor-info-main-headline item'}).getText()

    #Getting name of product and prices - fortunately both are spans
    smaller_soup = soup.find('div', {'class':'menu__items'}).findAll('span')
    temp_list= [elem.getText() for elem in smaller_soup]

    #Name will be every first, price will be every second element in smaller soup
    product_names = temp_list[::2]
    prices_temp = temp_list[1::2]
    restaurant_name = [restaurant for i in range(len(product_names))]

    #Price requires further alterations
    for elem in prices_temp:
        temp_elem = elem.split()
        prices.append("".join(temp_elem))

    #Put vectors into pandas dataframe    
    dict_for_df = {'Restaurant': restaurant_name, 'Product_Name': product_names, 'Price': prices}  
    df = pd.DataFrame(dict_for_df)
    
    final_df = final_df.append(df)

I have an issue below - will need to debug my code for restaurants where it failed. I was able to get at least 73 restaurants tough. 
Update: Not sure what caused the issue, will run this tomorrow.

In [301]:
final_df = pd.DataFrame(columns = ['Restaurant', 'Product_Name', 'Price']) 

for link in link_list:
    netpincer_parser(link)

ValueError: arrays must all be same length

In [302]:
final_df.describe()

,Restaurant,Product_Name,Price
count,8536,8536,8536
unique,73,7436,924
top,S.O.S. Night - Italházhozszállítás\n,Ketchup,innen2190Ft
freq,963,19,177


In [306]:
#final_df.to_csv(r'C:\Users\T450s\Python_directory\trial_netpincer.csv', 
#                           index = True, sep=',', encoding='utf-8')

# Make Restaurants DF 

### I think it would make sense to make a dataframe for restaurants separate from the itemised df above
#### The code to be generalised into a function + I need to solve for cases where we have 4 features only. 

In [3]:
restaurant_df = pd.DataFrame(columns = ['Restaurant', 'User-Rating',  'No-Ratings', 'Address', 'Feature1'
                                       , 'Feature2','Feature3', 'Feature4', 'Feature5']) 

In [10]:
def restaurant_parser(URL):
    
    global restaurant_df
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, "html.parser") 

    #Getting name of the restaurant
    restaurant = soup.find('div',{'class':'vendor-info-main-headline item'}).getText()

    #Getting features of restaurant
    features = soup.find('ul',{'class':'vendor-info-main-details-cuisines'}).findAll('li')
    features_list = [values.getText() for values in features[1:]]

    #Getting ratings of restaurants if available 
    ratings = soup.find('div',{'class':'ratings-component'}).findAll('span')
    ratings_list = [values.getText().split()for values in ratings[1:]]
        #Splitting out user rating out of how many
    ratings_list = [nums[0].split('/') for nums in ratings_list]
        #Breaking out lists in lists
    ratings_list = [[item] for sub_list in ratings_list for item in sub_list]

    #Getting the address of a Restaurant
    address = soup.find('p',{'class':'vendor-location'}).getText()

    #Putting all of the above in a dictionary
    dict_for_restaurant_df = {'Restaurant': restaurant, 'User-Rating':ratings_list[0][0],  'No-Ratings': ratings_list[2][0]
                              , 'Address' : address, 'Feature1': features_list[0], 'Feature2': features_list[1]
                              ,'Feature3': features_list[2], 'Feature4' : features_list[3],'Feature5': features_list[4]}

    #Add to restaurant_df
    restaurant_df = restaurant_df.append(dict_for_restaurant_df, ignore_index = True)

In [13]:
restaurant_parser(link_list[1])

IndexError: list index out of range

In [14]:
link_list[1]

'https://www.netpincer.hu//restaurant/b8fd/kiosk-pest'